In [19]:
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [2]:
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

In [3]:
con = engine.connect()
bookings = con.execute("SELECT * FROM Bookings")
bookings_df = pd.DataFrame(bookings.fetchall())
bookings_df.columns = bookings.keys()

facilities = con.execute("SELECT * FROM Facilities")
facilities_df = pd.DataFrame(facilities.fetchall())
facilities_df.columns = facilities.keys()

members = con.execute("SELECT * FROM Members")
members_df = pd.DataFrame(members.fetchall())
members_df.columns = members.keys()

In [68]:
bookings_df.head()
facilities_df.head()
#members_df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,Total_Revenue
0,0,Tennis Court 1,5.0,25.0,10000,200,4040.0
1,1,Tennis Court 2,5.0,25.0,8000,200,4205.0
2,2,Badminton Court,0.0,15.5,4000,50,604.5
3,3,Table Tennis,0.0,5.0,320,10,90.0
4,4,Massage Room 1,9.9,80.0,4000,3000,20807.9


In [5]:
members_df['joindate'] = members_df['joindate'].astype('datetime64[ns]')

In [6]:
members_df.set_index("memid", inplace=True)

In [7]:
bookings_df['starttime'] = bookings_df['starttime'].astype('datetime64[ns]')

In [8]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!



In [9]:
def calc_rev(members, facilities):
    facilities_df["Total_Revenue"] = 0 #clear "Total Revenue Column"
    
    for f in facilities: 
        
        for m in members:
            
            filt = (bookings_df.memid == m) & (bookings_df.facid == f)
            filt2 = bookings_df.starttime > members_df.loc[m].joindate
            book_filt = bookings_df[filt]
            bookings_filt2 = book_filt[filt2] 
            
            number_of_bookings = len(bookings_filt2)
            
            if m != 0:
                facilities_df.loc[f, "Total_Revenue"] += number_of_bookings * facilities_df.loc[f, "membercost"]
            else:
                facilities_df.loc[f, "Total_Revenue"] += number_of_bookings * facilities_df.loc[f, "guestcost"]
    
    

In [10]:
fac_list = list(facilities_df["facid"])
mem_list = list(members_df.index)

In [11]:
calc_rev(mem_list, fac_list)

/var/folders/0q/2w6362bn3v11r3hw5198hcvw0000gn/T/ipykernel_16508/3673374849.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bookings_filt2 = book_filt[filt2]


In [12]:
facilities_df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,Total_Revenue
0,0,Tennis Court 1,5.0,25.0,10000,200,4040.0
1,1,Tennis Court 2,5.0,25.0,8000,200,4205.0
2,2,Badminton Court,0.0,15.5,4000,50,604.5
3,3,Table Tennis,0.0,5.0,320,10,90.0
4,4,Massage Room 1,9.9,80.0,4000,3000,20807.9
5,5,Massage Room 2,9.9,80.0,4000,3000,6987.3
6,6,Squash Court,3.5,17.5,5000,80,4970.0
7,7,Snooker Table,0.0,5.0,450,15,115.0
8,8,Pool Table,0.0,5.0,400,15,265.0


In [6]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [28]:
members_df['recommendedby'].replace({'': np.nan}, inplace=True)

In [41]:
report = members_df[~members_df['recommendedby'].isna()][['surname','firstname','recommendedby']]

In [48]:
report.sort_values(["surname", "firstname"], ascending=[True, True], inplace=True)

In [49]:
report

,surname,firstname,recommendedby
memid,,,
15,Bader,Florence,9
12,Baker,Anne,9
16,Baker,Timothy,13
8,Boothe,Tim,3
5,Butters,Gerald,1
22,Coplin,Joan,16
36,Crumpet,Erica,2
7,Dare,Nancy,4
20,Genting,Matthew,5


In [7]:
#Q12: Find the facilities with their usage by member, but not guests */

In [60]:
facilities_usage = bookings_df[bookings_df['memid'] != 0].groupby("memid")

In [65]:
facilities_usage.get_group(2)

,bookid,facid,memid,starttime,slots
6,6,0,2,2012-07-04 09:00:00,3
7,7,0,2,2012-07-04 15:00:00,3
14,14,7,2,2012-07-04 14:00:00,2
15,15,8,2,2012-07-04 12:00:00,1
24,24,7,2,2012-07-05 18:30:00,2
...,...,...,...,...,...
3778,3778,2,2,2012-09-27 19:00:00,3
3815,3815,8,2,2012-09-27 08:30:00,1
3840,3840,2,2,2012-09-28 08:00:00,3
3918,3918,3,2,2012-09-29 08:30:00,2


In [8]:
#Q13: Find the facilities usage by month, but not guests */

In [80]:
bookings_df[bookings_df["starttime"].dt.month == 7]

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
...,...,...,...,...,...
653,653,8,7,2012-07-31 12:30:00,1
654,654,8,8,2012-07-31 13:30:00,1
655,655,8,6,2012-07-31 14:00:00,1
656,656,8,4,2012-07-31 17:00:00,1


In [102]:
fac_month_group = bookings_df.groupby(by=["facid", bookings_df.starttime.dt.month])

In [103]:
fac_month_group.get_group((3, 7))

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
19,19,3,3,2012-07-05 09:00:00,2
20,20,3,1,2012-07-05 19:00:00,2
30,30,3,1,2012-07-06 11:00:00,2
43,43,3,2,2012-07-07 12:30:00,2
58,58,3,1,2012-07-08 11:30:00,2
59,59,3,3,2012-07-08 18:30:00,2
60,60,3,1,2012-07-08 19:30:00,2
78,78,3,3,2012-07-09 17:00:00,2
79,79,3,3,2012-07-09 18:30:00,2


In [9]:
con.close()